In [18]:
import numpy as np
from data_process import *
import networkx as nx

# Cora

In [19]:
G = nx.Graph()
with open('data/cora/cora.content','r') as f:
    Lines=[x.strip().split('\t') for x in f]
for i in range(len(Lines)):
    lines = Lines[i]
    node = int(lines[0])
    features = list(map(int, lines[1:-1]))
    label = lines[-1]
    G.add_node(node, feature = features, label = label)

In [20]:
with open('data/cora/cora.cites','r') as f:
    Lines=[x.strip().split('\t') for x in f]
for i in range(len(Lines)):
    lines = Lines[i]
    G.add_edge(int(lines[0]), int(lines[1]), weight=1)

In [21]:
nx.write_gpickle(G, "data/cora.gpickle")

In [16]:
feat = nx.get_node_attributes(G, "feature")
import numpy as np
a = np.array(list(feat.values()))

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Homo-sapiens PPI 

In [ ]:
ppi = np.load("data/ppi/homo_sapiens.npz")

In [ ]:
G = nx.Graph()
G.add_edges_from(ppi["edge_list"])

In [ ]:
df = pd.DataFrame(ppi["group"])
num = np.arange(1, 51)
df.columns = [str(i) for i in np.arange(1, 51)]

In [ ]:
df['label'] = df.idxmax(axis=1)

In [ ]:
df.index.name = 'id'

In [ ]:
df

In [ ]:
nx.set_node_attributes(G, pd.Series(df.label, index=df.index).to_dict(), 'label')

In [ ]:
nx.set_edge_attributes(G, values = 1, name = 'weight')

In [ ]:
nx.write_gpickle(G, "data/ppi.gpickle")

# CiteSeer

In [ ]:
f = open("data/citeseer/citeseer.content", "r")
Lines = f.readlines()

In [ ]:
labels = {}
for line in Lines:
    labels[line.split()[0]] = line.split()[-1]

In [ ]:
f = open("data/citeseer/citeseer.cites", "r")
Lines = f.readlines()

In [ ]:
edges = []
for line in Lines:
    edges.append([line.split()[0], line.split()[1]])

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
nx.set_node_attributes(G, labels, "label")
nx.set_edge_attributes(G, values = 1, name = 'weight')

In [ ]:
nx.write_gpickle(G, "data/citeseer.gpickle")

# Pubmed Diabetes

In [ ]:
with open('data/pubmed/Pubmed-Diabetes.NODE.paper.tab','r') as f:
    Lines=[x.strip().split('\t') for x in f]

In [ ]:
labels = {}
for line in Lines[2:]:
    labels[line[0]] = line[1][line[1].find("=")+1:]

In [ ]:
with open('data/pubmed/Pubmed-Diabetes.DIRECTED.cites.tab','r') as f:
    Lines=[x.strip().split('\t') for x in f]

In [ ]:
edges = []
for line in Lines[2:]:
    edges.append([line[1][line[1].find(":")+1:], line[3][line[3].find(":")+1:]])

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
nx.set_node_attributes(G, labels, "label")
nx.set_edge_attributes(G, values = 1, name = 'weight')

In [ ]:
nx.write_gpickle(G, "data/pubmed.gpickle")

# Amazon Co-Purchase

In [ ]:
computers = np.load("data/amazon/amazon_electronics_computers.npz")

In [ ]:
# import dgl
# dataset = dgl.data.AmazonCoBuyComputerDataset()

In [ ]:
G = nx.Graph(dgl.to_networkx(dataset[0], node_attrs=graph.ndata, edge_attrs=graph.edata))

In [ ]:
for k in nx.get_node_attributes(G, 'label').keys():
    label = nx.get_node_attributes(G, 'label')[k].cpu().detach().numpy().flatten()[0]
    G.nodes[k]['label'] = label

In [ ]:
nx.write_gpickle(G, "data/amazon.gpickle")

# BlogCatalog3

In [ ]:
file = open("data/blogcatalog/edges.csv", "r")

In [ ]:
csvreader = csv.reader(file)

In [ ]:
edges = []
for row in csvreader:
    edges.append(row)

In [ ]:
file = open("data/blogcatalog/group-edges.csv", "r")

In [ ]:
csvreader = csv.reader(file)

In [ ]:
labels = {}
for row in csvreader:
    labels[row[0]] = row[1]

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
nx.set_node_attributes(G, labels, "label")
nx.set_edge_attributes(G, values = 1, name = 'weight')

In [ ]:
nx.write_gpickle(G, "data/blogcatalog.gpickle")

# Github

In [ ]:
github = np.load("data/github/github.npz")

In [ ]:
G = nx.Graph()
G.add_edges_from(github["edges"])

In [ ]:
df = pd.DataFrame(github["target"])
df.columns = ["label"]

In [ ]:
features = pd.DataFrame(pd.np.column_stack([github["features"]])).T.to_dict('list')

In [ ]:
nx.set_node_attributes(G, pd.Series(df.label, index=df.index).to_dict(), 'label')

In [ ]:
nx.set_node_attributes(G, features, "features")

In [ ]:
nx.set_edge_attributes(G, values = 1, name = 'weight')

In [ ]:
nx.write_gpickle(G, "data/github.gpickle")